<a href="https://colab.research.google.com/github/jakubpiwowarski/Personal_protective_equipment_detector/blob/main/Poker_Hand_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab.patches import cv2_imshow
!pip install ultralytics==8.0.28

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

In [ ]:
pip install cvzone


In [ ]:
import cvzone
import math
import cv2

In [ ]:
!pip install opencv-python-headless
!pip install google-colab

In [ ]:
pip install filterpy==1.4.5

In [ ]:
pip install scikit-image

In [ ]:
pip install lap==0.4.0

In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector')

In [ ]:
sys.path.append('/content/drive/MyDrive/101 - Object Detection /Pliki z kursu/Project 1 - Car Counter/sort.py')

In [22]:
sys.path.append('/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/pokerHandDetectorFunctions.py')

In [ ]:
!cp -r "/content/drive/MyDrive/101-Object_Detection/Pliki/Project 1 - Car Counter/sort.py" '/content/'

In [23]:
!cp -r "/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/pokerHandDetectorFunctions.py" '/content/'

In [ ]:
from sort import *

In [24]:
import pokerHandDetectorFunctions

In [17]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [18]:
model= YOLO('/content/drive/MyDrive/101-Object_Detection/Pliki/Project_4-Poker_Hand_Detector/playingCards.pt')

In [ ]:
classNames = ['10C', '10D', '10H', '10S', '2C', '2D', '2H', '2S',
              '3C', '3D', '3H', '3S', '4C', '4D', '4H', '4S',
              '5C', '5D', '5H', '5S', '6C', '6D', '6H', '6S',
              '7C', '7D', '7H', '7S', '8C', '8D', '8H', '8S',
              '9C', '9D', '9H', '9S', 'AC', 'AD', 'AH', 'AS',
              'JC', 'JD', 'JH', 'JS', 'KC', 'KD', 'KH', 'KS',
              'QC', 'QD', 'QH', 'QS']


In [25]:
hand =[]

In [ ]:
while True:
    success, img = cap.read()
    results = model(img,stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            #bounding box
            x1,y1,x2,y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2-x1, y2-y1
            bbox = int(x1), int(y1), int(x2), int(y2)
            cvzone.cornerRect(img,(x1,y1,w,h))
            #confidence
            conf = math.ceil((box.conf[0]*100))/100
            print(conf)

            #clas name
            cls = int(box.cls[0])
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)),scale=2,thickness=2)

    if len(hand) == 5:
      results = pokerHandDetectorFunctions.findPokerHand(hand)

    cv2_imshow(img)
    cv2.waitKey(1)
